In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

# Generating Data

In [425]:
'''
Cases 1,2 without symmetric connectivity
Focus on standard deviation (variance)
Focus on second moment (self)
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,10,5,10
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.1,1.0,10) # a series of symmetricity value
print('symetaseries:',symetaseries)
etaset=0*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.00,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
meanReigAmE,meanReigAmI=np.mean(meigvecAm[:NE,0]),np.mean(meigvecAm[NE:,0])
meanLeigAmE,meanLeigAmI=np.mean(neigvecAm[:NE,0])/eigvAm[0],np.mean(neigvecAm[NE:,0])/eigvAm[0]
'''
for reference
'''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
'''
inverse S
'''
invB1=np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
X1,Y1=meigvecAm[:,1:].copy(),neigvecAm[:,1:].copy()
SAm = X1@invB1@Y1.T
invSAm=la.inv(SAm)

'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
'''
>>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>>
'''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.5,0.5 ## HETEROgeneous random gain
# xee,xei,xie,xii=1.0,0.5,0.1,0.8 ## HETEROgeneous random gain

gderiv = np.zeros((NE*2,NE*2))
gderiv[:NE,:NE],gderiv[:NE,NE:],gderiv[NE:,:NE],gderiv[NE:,NE:]=xee/np.sqrt(2*NE),xei/np.sqrt(2*NE),xie/np.sqrt(2*NE),xii/np.sqrt(2*NE)

'''
Recording Variables
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,ntrial,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2,N)),np.zeros((ngavg,ntrial,NE*2,N))
Beigvseries=np.zeros((ngavg,ntrial,N))
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2)),np.zeros((ngavg,ntrial,3,2)) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
coveigvecAmseries=np.zeros((ngavg,ntrial,3),dtype=complex)
''' All based on perturbation theorem '''
Rvecperturb,Lvecperturb=np.zeros((ngavg,ntrial,NE*2)),np.zeros((ngavg,ntrial,NE*2))
covefftheo=np.zeros((ngavg,ntrial,3))

''' Iterative Processing '''
gaverageseries=np.linspace(0.1,1.0,ngavg)
for idxeta,eta in enumerate(gaverageseries):
    for iktrial in range(ntrial):
        etaset=0.0*np.ones(3)#eta*np.ones(3)
        # etaset[2]*=0.5
        etaset[1]*=0.5
        gaverage=eta
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        # ''' heterogeneous ETA method 1 '''
        # ## EE ##
        # asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownEE,idyDownEE]=aamp*XsymT[idxDownEE,idyDownEE].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownEE,idyDownEE].copy()
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*aamp*XsymT[idxUpEE,idyUpEE].copy()+signeta[0]*np.sqrt(1-aamp**2)*Xsym[idxUpEE,idyUpEE].copy()
        # ## EI IE##
        # asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[NE:,:NE]=aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        # Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+signeta[1]*np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        # ## II ##
        # asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownII,idyDownII]=aamp*XsymT[idxDownII,idyDownII].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownII,idyDownII].copy()
        # Xinit[idxUpII,idyUpII]=signeta[2]*aamp*XsymT[idxUpII,idyUpII].copy()+signeta[2]*np.sqrt(1-aamp**2)*Xsym[idxUpII,idyUpII].copy()

        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        # X0=Xinit.copy()*gaverage/np.sqrt(2)
        # X=aamp*X0.T+np.sqrt(1-aamp**2)*X0

        # ''' HETEROGENEOUS ETA method 2 '''
        # # EE#
        # Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # # EI/IE#
        # Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        # Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # # EE#
        # Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        # Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])

        # # ''' homogeneous '''
        # X=Xinit.copy()
        # X*=(gaverage/np.sqrt(2))

        # ''' heterogeneous '''
        X=Xinit.copy()
        X[:NE,:NE]*=(xee*gaverage/np.sqrt(2))
        X[:NE,NE:]*=(xei*gaverage/np.sqrt(2))
        X[NE:,:NE]*=(xie*gaverage/np.sqrt(2))
        X[NE:,NE:]*=(xii*gaverage/np.sqrt(2))
        '''
        theory cov
        '''
        l0EXX=np.squeeze(y0.T@X@X)
        # print(np.shape(l0EXX))
        covefftheo[idxeta,iktrial,0]=np.sum(l0EXX[:NE]*x0[:NE,0])
        covefftheo[idxeta,iktrial,1]=np.sum(l0EXX[NE:]*x0[NE:,0])
        covefftheo[idxeta,iktrial,2]=np.sum(l0EXX*x0)
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        symavg/=(2*NE)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        # print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symtheofull,'; num:',symnumfull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]

        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()
        

        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
        if (np.mean(tildex[:NE])*x0[0])<0:
            tildex*=(-1)
        if (np.mean(tildey[:NE])*y0[0,0])<0:
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        # ynorm0_=np.reshape(ynorm0.copy(),(NE*2,1))
        # '''
        # calculate ynorm
        # '''

        # ynorm0modi=-1*invSAm@ynorm0_
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:].copy(),ynorm0[:].copy()
        

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        coveigvecAmseries[idxeta,iktrial,0]=np.reshape(reig[:NE,0]-meanReigAmE,(NE,1)).T@np.reshape(leig[:NE,0]-meanLeigAmE,(NE,1))
        coveigvecAmseries[idxeta,iktrial,1]=np.reshape(reig[NE:,0]-meanReigAmI,(NE,1)).T@np.reshape(leig[NE:,0]-meanLeigAmI,(NE,1))
        coveigvecAmseries[idxeta,iktrial,2]=coveigvecAmseries[idxeta,iktrial,0]+coveigvecAmseries[idxeta,iktrial,1]


symetaseries: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalization m & n
1.000000000000003 5.147815070493495
eigenvalues: [-2.5+0.j]
eta full: 0.0 ; num: 4.486591416674966e-05
eta full: 0.0 ; num: -1.985249006460663e-05
eta full: 0.0 ; num: 5.890850071906183e-06
eta full: 0.0 ; num: 6.567394705777453e-06
eta full: 0.0 ; num: 1.4053597385329391e-05
eta full: 0.0 ; num: 2.6023396527120463e-07
eta full: 0.0 ; num: 1.3445578263705819e-05
eta full: 0.0 ; num: 3.258333488580166e-05
eta full: 0.0 ; num: 6.436413537417108e-06
eta full: 0.0 ; num: 2.4923260605202137e-05
eta full: 0.0 ; num: 3.4873309174594803e-05
eta full: 0.0 ; num: 7.89906346292537e-05
eta full: 0.0 ; num: 3.570729587446075e-05
eta full: 0.0 ; num: 3.077953662928224e-05
eta full: 0.0 ; num: 2.0517557507618697e-05
eta full: 0.0 ; num: -2.4542487217219484e-05
eta full: 0.0 ; num: 0.00011309249799723394
eta full: 0.0 ; num: 0.00010958868220517403
eta full: 0.0 ; num: -2.316014342080617e-05
eta full: 0.0 ; num: 5.9411436938696

In [426]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
# ## calculating the statistics of perturbation lambda
# # Coding uses only one point
# std_perturb_Lambda_num = np.std(eigvalseries[ngavg-1,:,0])
# std_perturb_Lambda_theo = np.sqrt((JE**2*(xee+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
# ## theoretical curve for standard deviation of the eigenvalue
# std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries

# print('std of perturbation lambda, num:',std_perturb_Lambda_num,'; theo:',std_perturb_Lambda_theo)
# '''
# Variance of the perturbations in eigenvalue lambda
# \delta lambda =\Delta lambda = (1) unit change in g
# '''
# std_perturbLambda = np.std(eigvalseries[:,:,0],axis=1)
# fig,ax=plt.subplots(figsize=(4,4))
# cm='rbg'
# ax.plot(gaverageseries,std_perturbLambda,color=cm[2],linewidth=1.5,label='numerical')
# # polyfitting
# p=np.polyfit(gaverageseries,std_perturbLambda,1)
# std_perturbLambda_linear = np.zeros(ngavg)
# std_perturbLambda_linear = p[0]*gaverageseries+p[1]
# ax.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
# ax.plot(gaverageseries,std_perturbLambda_linear_theo,color='darkgreen',linewidth=1.5,label='analytical')
# print('std delta_lambda numerical:',p[0])

# '''
# plt and ax
# '''
# ylims=[-0.00,0.08]
# xlims=[-0.0,1.1]
# yticks = np.linspace(0.0,0.08,2)
# xticks = np.linspace(0.0,1.0,2)
# ax.set_xlim(xlims)
# ax.set_xticks(xticks)
# ax.set_ylim(ylims)
# ax.set_yticks(yticks)
# # ax.legend()

# '''
# right eigenvector
# '''
# ## calculating the statistics of perturbation lambda
# # Coding uses only one point
# std_perturb_Reig_num,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
# std_perturb_Reig_num[0],std_perturb_Reig_num[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
# std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
# ## theoretical curve for standard deviation of the eigenvalue
# std_perturbReig_linear_theo=np.zeros((ngavg,2))
# std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
# std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries

# print('std of perturbation right eigenvector, num:',std_perturb_Reig_num,'; theo:',std_perturb_Reig_theo)
# '''
# Variance of the perturbations in eigenvalue lambda
# \delta lambda =\Delta lambda = (1) unit change in g
# '''
# std_perturbReig,std_perturbReig_linear=np.zeros((ngavg,2)),np.zeros((ngavg,2))
# std_perturbReig[:,0],std_perturbReig[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)
# fig,ax=plt.subplots(2,1,figsize=(4,6))
# cm='rbg'
# for i in range(2):
#     # if i>0:
#     #     i-=1
#     ax[i].plot(gaverageseries,std_perturbReig[:,i],color=cm[i],linewidth=1.5,label='numerical')
#     # polyfitting
#     p=np.polyfit(gaverageseries,std_perturbReig[:,i],1)
#     print('p,',p)
#     std_perturbReig_linear[:,i] = p[0]*gaverageseries+p[1]
#     ax[i].plot(gaverageseries,std_perturbReig_linear[:,i],color=cm[i],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
#     print('std delta_lambda numerical:',p[0])


#     # '''
#     # plt and ax
#     # '''
#     # ylims=[-0.00,0.004]
#     # xlims=[-0.0,1.1]
#     # yticks = np.linspace(0.0,0.004,2)
#     # xticks = np.linspace(0.0,1.0,2)
#     # ax[i].set_xlim(xlims)
#     # ax[i].set_xticks(xticks)
#     # ax[i].set_ylim(ylims)
#     # ax[i].set_yticks(yticks)
#     # # ax.legend()
# ax[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='darkred',linewidth=1.5,label='analytical')
# ax[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='darkblue',linewidth=1.5,label='analytical')
# # ax.legend()


'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_num,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_num[0],std_perturb_Leig_num[1] = np.mean(statesLeig[ngavg-1,:,0,1])/eigvAm[0],np.mean(statesLeig[ngavg-1,:,1,1])/eigvAm[0]
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo=np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_num,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturbLeig,std_perturbLeig_linear=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig[:,0],std_perturbLeig[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)
# fig,ax=plt.subplots(2,1,figsize=(4,6))
cm='mg'#'rb'#
for i in range(2):
    # if i>0:
    #     i-=1
    ax[i].plot(gaverageseries,std_perturbLeig[:,i],color=cm[i],linewidth=1.5,label='numerical')
    # polyfitting
    p=np.polyfit(gaverageseries,std_perturbLeig[:,i],1)
    print('p,',p)
    std_perturbLeig_linear[:,i] = p[0]*gaverageseries+p[1]
    ax[i].plot(gaverageseries,std_perturbLeig_linear[:,i],color=cm[i],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
    print('std delta_lambda numerical:',p[0])


    '''
    plt and ax
    '''
    ylims=[-0.00,0.02]
    xlims=[-0.0,1.1]
    yticks = np.linspace(0.0,0.02,2)
    xticks = np.linspace(0.0,1.0,2)
    ax[i].set_xlim(xlims)
    ax[i].set_xticks(xticks)
    ax[i].set_ylim(ylims)
    ax[i].set_yticks(yticks)
    # ax.legend()
ax[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='darkred',linewidth=1.5,label='analytical')
ax[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='darkblue',linewidth=1.5,label='analytical')



std of perturbation left eigenvector, num: [-0.00924557 -0.00931967] ; theo: [0.02126656 0.02126656]
p, [ 0.02353067 -0.00061061]
std delta_lambda numerical: 0.023530668274984814
p, [ 0.02364652 -0.00064836]
std delta_lambda numerical: 0.02364651996381993


In [408]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
mean_perturb_Lambda_num = np.mean(eigvalseries[:,:,0],axis=1)
mean_perturb_Lambda_theo = np.ones(ngavg)*eigvAm[0]
fig,ax=plt.subplots(figsize=(4,4))
cm='rbg'
ax.plot(gaverageseries,mean_perturb_Lambda_num,color=cm[0],linewidth=1.5,label='numerical')
ax.plot(gaverageseries,mean_perturb_Lambda_theo,color=cm[0],linewidth=1.5,linestyle='--',label='numerical')


'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
mean_perturb_Reig_num,mean_perturb_Reig_theo=np.zeros((ngavg,2)),np.zeros((ngavg,2))
mean_perturb_Reig_num[:,0],mean_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,0],axis=1),np.mean(statesReig[:,:,1,0],axis=1)
mean_perturb_Reig_theo[:,0],mean_perturb_Reig_theo[:,1] = np.ones(ngavg)*x0[0],np.ones(ngavg)*x0[NE]
fig,ax=plt.subplots(2,1,figsize=(4,6))
cm='rbg'
for i in range(2):
    ax[i].plot(gaverageseries,mean_perturb_Reig_num[:,i],color=cm[0],linewidth=1.5,label='numerical')
    ax[i].plot(gaverageseries,mean_perturb_Reig_theo[:,i],color=cm[0],linewidth=1.5,linestyle='--',label='numerical') 


'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
mean_perturb_Leig_num,mean_perturb_Leig_theo=np.zeros((ngavg,2)),np.zeros((ngavg,2))
mean_perturb_Leig_num[:,0],mean_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,0],axis=1),np.mean(statesLeig[:,:,1,0],axis=1)
mean_perturb_Leig_theo[:,0],mean_perturb_Leig_theo[:,1] = np.ones(ngavg)*y0[0],np.ones(ngavg)*y0[NE]
fig,ax=plt.subplots(2,1,figsize=(4,6))
cm='rbg'
for i in range(2):
    ax[i].plot(gaverageseries,mean_perturb_Leig_num[:,i],color=cm[0],linewidth=1.5,label='numerical')
    ax[i].plot(gaverageseries,mean_perturb_Leig_theo[:,i],color=cm[0],linewidth=1.5,linestyle='--',label='numerical') 



In [424]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii
# '''
# # original data
# eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
# Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,N)),np.zeros((neta,ntrial,NE*2,N))
# Beigvseries=np.zeros((neta,ntrial,N))

# statesReig,statesLeig=np.zeros((neta,ntrial,3,2)),np.zeros((neta,ntrial,3,2)) #E,I,Total/mean,std# 
# coveigvecseries=np.zeros((neta,ntrial,3),dtype=complex) # E, I and total
# '''
# # np.savez(now+'_perturbationHomeeta_data',Jparameters=Jparameters,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=Beigvseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries)

np.savez(now+'_perturbationZeroeta_data',x0=x0,y0=y0,Jparameters=Jparameters,Randgain=Randgain,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries)

# # np.savez(now+'_perturbationZeroeta_data',Jparameters=Jparameters,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm[0],statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries)

In [451]:
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
'''
# generate mean matrix
nrank=1
Nt=np.array([500,500])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,10,5,10
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.1,1.0,10) # a series of symmetricity value
print('symetaseries:',symetaseries)
etaset=0*np.ones(3)
# etaset[1]=0.4
# etaset[2]=0.6
'''
## Three \bar{J} cases
'''
JE,JI,a,b=3.50,1.0,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
# JI,JE,a,b=3.5,1.0,3.5,1.0 # already recording -3.5# 2.0
# JI,JE=0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
meanReigAmE,meanReigAmI=np.mean(meigvecAm[:NE,0]),np.mean(meigvecAm[NE:,0])
meanLeigAmE,meanLeigAmI=np.mean(neigvecAm[:NE,0])/eigvAm[0],np.mean(neigvecAm[NE:,0])/eigvAm[0]
'''
for reference
'''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
'''
inverse S
'''
invB1=np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    invB1[i,i]=1/(eigvAm[i+1]-eigvAm[0])
X1,Y1=meigvecAm[:,1:].copy(),neigvecAm[:,1:].copy()
SAm = X1@invB1@Y1.T

'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
'''
>>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>>
'''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
gderiv = np.zeros((NE*2,NE*2))
gderiv[:NE,:NE],gderiv[:NE,NE:],gderiv[NE:,:NE],gderiv[NE:,NE:]=xee/np.sqrt(2*NE),xei/np.sqrt(2*NE),xie/np.sqrt(2*NE),xii/np.sqrt(2*NE)

'''
Recording Variables
'''
# original data
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,ntrial,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2,N)),np.zeros((ngavg,ntrial,NE*2,N))
Beigvseries=np.zeros((ngavg,ntrial,N))
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2)),np.zeros((ngavg,ntrial,3,2)) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
coveigvecAmseries=np.zeros((ngavg,ntrial,3),dtype=complex)
''' All based on perturbation theorem '''
Rvecperturb,Lvecperturb=np.zeros((ngavg,ntrial,NE*2)),np.zeros((ngavg,ntrial,NE*2))
covefftheo=np.zeros((ngavg,ntrial,3))

''' Iterative Processing '''
gaverageseries=np.linspace(0.1,1.0,ngavg)
for idxeta,eta in enumerate(gaverageseries):
    for iktrial in range(ntrial):
        etaset=1.0*np.ones(3)#eta*np.ones(3)
        # etaset[2]*=0.5
        etaset[1]*=0.6
        gaverage=eta
        Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        XsymT=Xsym.copy().T
        Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
        Xinit = Xsym.copy()
        # ''' heterogeneous ETA method 1 '''
        # ## EE ##
        # asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownEE,idyDownEE]=aamp*XsymT[idxDownEE,idyDownEE].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownEE,idyDownEE].copy()
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*aamp*XsymT[idxUpEE,idyUpEE].copy()+signeta[0]*np.sqrt(1-aamp**2)*Xsym[idxUpEE,idyUpEE].copy()
        # ## EI IE##
        # asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[NE:,:NE]=aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        # Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+signeta[1]*np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        # ## II ##
        # asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        # aamp=np.sqrt(asqr)
        # Xinit[idxDownII,idyDownII]=aamp*XsymT[idxDownII,idyDownII].copy()+np.sqrt(1-aamp**2)*Xsym[idxDownII,idyDownII].copy()
        # Xinit[idxUpII,idyUpII]=signeta[2]*aamp*XsymT[idxUpII,idyUpII].copy()+signeta[2]*np.sqrt(1-aamp**2)*Xsym[idxUpII,idyUpII].copy()

        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1+np.sqrt(1-etaset[0]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1+np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1+np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        # X0=Xinit.copy()*gaverage/np.sqrt(2)
        # X=aamp*X0.T+np.sqrt(1-aamp**2)*X0

        # ''' HETEROGENEOUS ETA method 2 '''
        # # EE#
        # Xinit[idxDownEE,idyDownEE]=Xsym[idxDownEE,idyDownEE]*np.sqrt(etaset[0])+Yidendown[idxDownEE,idyDownEE].copy()*np.sqrt(1-etaset[0])
        # Xinit[idxUpEE,idyUpEE]=signeta[0]*XsymT[idxUpEE,idyUpEE]*np.sqrt(etaset[0])+Yidenup[idxUpEE,idyUpEE].copy()*np.sqrt(1-etaset[0])
        # # EI/IE#
        # Xinit[NE:,:NE]=Xsym[NE:,:NE]*np.sqrt(etaset[1])+Yidendown[NE:,:NE].copy()*np.sqrt(1-etaset[1])
        # Xinit[:NE,NE:]=signeta[1]*XsymT[:NE,NE:]*np.sqrt(etaset[1])+Yidenup[:NE,NE:].copy()*np.sqrt(1-etaset[1])
        # # EE#
        # Xinit[idxDownII,idyDownII]=Xsym[idxDownII,idyDownII]*np.sqrt(etaset[2])+Yidendown[idxDownII,idyDownII].copy()*np.sqrt(1-etaset[2])
        # Xinit[idxUpII,idyUpII]=signeta[2]*XsymT[idxUpII,idyUpII]*np.sqrt(etaset[2])+Yidenup[idxUpII,idyUpII].copy()*np.sqrt(1-etaset[2])
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])
        # print(etaset[0]*signeta[0],etaset[2]*signeta[2],2*etaset[1]*signeta[1])

        # # ''' homogeneous '''
        # X=Xinit.copy()
        # X*=(gaverage/np.sqrt(2))

        # ''' heterogeneous '''
        X=Xinit.copy()
        X[:NE,:NE]*=(xee*gaverage/np.sqrt(2))
        X[:NE,NE:]*=(xei*gaverage/np.sqrt(2))
        X[NE:,:NE]*=(xie*gaverage/np.sqrt(2))
        X[NE:,NE:]*=(xii*gaverage/np.sqrt(2))
        '''
        theory cov
        '''
        l0EXX=np.squeeze(y0.T@X@X)
        # print(np.shape(l0EXX))
        covefftheo[idxeta,iktrial,0]=np.sum(l0EXX[:NE]*x0[:NE,0])
        covefftheo[idxeta,iktrial,1]=np.sum(l0EXX[NE:]*x0[NE:,0])
        covefftheo[idxeta,iktrial,2]=np.sum(l0EXX*x0)
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        '''
        Adding extra value mean-value can not be neglected
        '''
        barmE,barmI,barnE,barnI=meigvecAm[0,0],meigvecAm[NE,0],neigvecAm[0,0],neigvecAm[NE,0]
        # the first term
        symavg = (barmE*barnE*NE)**2+(barmI*barnI*NE)**2+(barmE*barnI*barmI*barnE)*(NE**2)+(barmE*barnI*barmI*barnE)*(NE**2)
        symavg/=(2*NE)
        avgmean=np.mean(np.mean(Am))**2*((2*NE)**2)
        # print('compare the first and the last, mean, terms:',symavg,avgmean)
        print('eta full:',symtheofull,'; num:',symnumfull)
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]

        eigvalseries[idxeta,iktrial,0],mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=eigvJ[0],meig[:,0].copy(),neig[:,0].copy()
        

        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=neig.copy()/normval.copy()
        '''
        normalization 0 (reference)
        '''
        tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
        if (np.mean(tildex[:NE])*x0[0,0])<0:
            tildex*=(-1)
        if (np.mean(tildey[:NE])*y0[0,0])<0:
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:].copy(),ynorm0[:].copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        coveigvecAmseries[idxeta,iktrial,0]=np.reshape(reig[:NE,0]-meanReigAmE,(NE,1)).T@np.reshape(leig[:NE,0]-meanLeigAmE,(NE,1))
        coveigvecAmseries[idxeta,iktrial,1]=np.reshape(reig[NE:,0]-meanReigAmI,(NE,1)).T@np.reshape(leig[NE:,0]-meanLeigAmI,(NE,1))
        coveigvecAmseries[idxeta,iktrial,2]=coveigvecAmseries[idxeta,iktrial,0]+coveigvecAmseries[idxeta,iktrial,1]


symetaseries: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalization m & n
1.0000000000000004 5.14781507049352
eigenvalues: [2.5+0.j]
eta full: 0.008000000000000002 ; num: 0.008030486375518696
eta full: 0.008000000000000002 ; num: 0.00797901111281415
eta full: 0.008000000000000002 ; num: 0.007991508342721337
eta full: 0.008000000000000002 ; num: 0.008024108837012597
eta full: 0.008000000000000002 ; num: 0.007937831249309328
eta full: 0.008000000000000002 ; num: 0.008016511251702505
eta full: 0.008000000000000002 ; num: 0.00794303373990028
eta full: 0.008000000000000002 ; num: 0.008006071590774102
eta full: 0.008000000000000002 ; num: 0.008040857333635519
eta full: 0.008000000000000002 ; num: 0.007909335850665026
eta full: 0.03200000000000001 ; num: 0.03205318054972704
eta full: 0.03200000000000001 ; num: 0.03204316256118347
eta full: 0.03200000000000001 ; num: 0.032023430910981476
eta full: 0.03200000000000001 ; num: 0.03204282692532922
eta full: 0.03200000000000001 ; num: 0.032053606

In [453]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii
# '''
# # original data
# eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank),dtype=complex)
# Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,N)),np.zeros((neta,ntrial,NE*2,N))
# Beigvseries=np.zeros((neta,ntrial,N))

# statesReig,statesLeig=np.zeros((neta,ntrial,3,2)),np.zeros((neta,ntrial,3,2)) #E,I,Total/mean,std# 
# coveigvecseries=np.zeros((neta,ntrial,3),dtype=complex) # E, I and total
# '''
# # np.savez(now+'_perturbationHomeeta_data',Jparameters=Jparameters,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=Beigvseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries)

np.savez(now+'_perturbationHETEReta_data',x0=x0,y0=y0,Jparameters=Jparameters,Randgain=Randgain,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries)

# # np.savez(now+'_perturbationZeroeta_data',Jparameters=Jparameters,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm[0],statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,coveigvecAmseries=coveigvecAmseries)

In [448]:
'''
* Loading data
    Picking up data:
    gaverageseries,eigvAm,eigvalseries
    statesReig,statesLeig
'''
strfile='2021-03-09-21_15_43_perturbationHETEReta_data' # 1,-0.6,1 
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
Jparameters=data['Jparameters']
print('scale:',Jparameters[4])
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
x0,y0=data['x0'],data['y0']
print('eta:',etaset,'; signs:',signeta)

scale: 500.0
JEE: 3.5
eta: [1.  0.6 1. ] ; signs: [ 1. -1.  1.]


In [452]:
'''
### >>>>>>>>>> Cases with Symmetry >>>>>>>>>>>>>>>>
Theoretical Values to be Tested
  statistics of elements in right and left eigenvectors
  eigenvalues: meaneigval_theo,

  eigenvectors:
    Right Eigenvector: meanReigvec_theo[0](E), meanReigvec_theo[1](I)
    Left Eigenvector:  meanLeigvec_theo[0](E), meanLeigvec_theo[1](I)

  noise covariance:
    noisecov_theo[0](E), noisecov_theo[1](I)
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''

meanReigvec_theo,meanLeigvec_theo=np.zeros((ngavg,2)),np.zeros((ngavg,2))
meaneigval_theo=np.zeros(ngavg)
noisecov_theo=np.zeros((ngavg,2))
## comparable numerical solutions
meanReigvec_num,meanLeigvec_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
meaneigval_num = np.zeros(ngavg)
noisecov_num=np.zeros((ngavg,2))
## polyfit curve
meanReigvec_poly,meanLeigvec_poly=np.zeros((ngavg,2)),np.zeros((ngavg,2))
meaneigval_poly=np.zeros(ngavg)

### Statistics of Right Eigenvectors
meanReigvec_theo[:,0],meanReigvec_theo[:,1]=x0[0]-x0[0]*(etaset[0]*signeta[0]+etaset[1]*signeta[1])*gaverageseries**2/(4.0*eigvAm[0]**2),\
x0[NE+1]-x0[NE+1]*(etaset[2]*signeta[2]+etaset[1]*signeta[1])*gaverageseries**2/(4.0*eigvAm[0]**2)
meanReigvec_num[:,0],meanReigvec_num[:,1]=np.mean(statesReig[:,:,0,0],axis=1),np.mean(statesReig[:,:,1,0],axis=1)
cm='rmby'
figR,axR=plt.subplots(2,1,figsize=(5,4))
axR[0].plot(gaverageseries,meanReigvec_num[:,0],color=cm[0],linewidth=1.5)
axR[0].plot(gaverageseries,meanReigvec_theo[:,0],color=cm[1],linewidth=1.5)
axR[1].plot(gaverageseries,meanReigvec_num[:,1],color=cm[2],linewidth=1.5)
axR[1].plot(gaverageseries,meanReigvec_theo[:,1],color=cm[3],linewidth=1.5)
### polyfitting
pRE=np.polyfit(gaverageseries,meanReigvec_num[:,0],2)
meanReigvec_poly[:,0]=pRE[0]*gaverageseries**2+pRE[1]*gaverageseries+pRE[2]
pRI=np.polyfit(gaverageseries,meanReigvec_num[:,1],2)
meanReigvec_poly[:,1]=pRI[0]*gaverageseries**2+pRI[1]*gaverageseries+pRI[2]
axR[0].plot(gaverageseries,meanReigvec_poly[:,0],color=cm[0],linestyle='--',linewidth=1.,alpha=0.5)
axR[1].plot(gaverageseries,meanReigvec_poly[:,1],color=cm[2],linestyle='--',linewidth=1.,alpha=0.5)

### Statistics of Right Eigenvectors
norm_stdE=y0[0]*(etaset[0]*signeta[0]+etaset[1]*signeta[1])*gaverageseries**2/(4.0*eigvAm[0]**2)
norm_stdI=y0[NE+1]*(etaset[2]*signeta[2]+etaset[1]*signeta[1])*gaverageseries**2/(4.0*eigvAm[0]**2)
meanLeigvec_theo[:,0],meanLeigvec_theo[:,1]=y0[0]-norm_stdE,y0[NE+1]-norm_stdI
meanLeigvec_num[:,0],meanLeigvec_num[:,1]=np.mean(statesLeig[:,:,0,0],axis=1),np.mean(statesLeig[:,:,1,0],axis=1)
figL,axL=plt.subplots(2,1,figsize=(5,4))
axL[0].plot(gaverageseries,meanLeigvec_num[:,0],color=cm[0],linewidth=1.5)
axL[0].plot(gaverageseries,meanLeigvec_theo[:,0],color=cm[1],linewidth=1.5)
axL[1].plot(gaverageseries,meanLeigvec_num[:,1],color=cm[2],linewidth=1.5)
axL[1].plot(gaverageseries,meanLeigvec_theo[:,1],color=cm[3],linewidth=1.5)
### polyfitting
pRE=np.polyfit(gaverageseries,meanLeigvec_num[:,0],2)
meanLeigvec_poly[:,0]=pRE[0]*gaverageseries**2+pRE[1]*gaverageseries+pRE[2]
pRI=np.polyfit(gaverageseries,meanLeigvec_num[:,1],2)
meanLeigvec_poly[:,1]=pRI[0]*gaverageseries**2+pRI[1]*gaverageseries+pRI[2]
axL[0].plot(gaverageseries,meanLeigvec_poly[:,0],color=cm[0],linestyle='--',linewidth=1.,alpha=0.5)
axL[1].plot(gaverageseries,meanLeigvec_poly[:,1],color=cm[2],linestyle='--',linewidth=1.,alpha=0.5)

### Statistics of eigenvalue outlier
etasum = np.sum(signeta*etaset)+signeta[1]*etaset[1]
meaneigval_theo = eigvAm[0]+gaverageseries**2*etasum/4/eigvAm[0]
meaneigval_num=np.mean(eigvalseries[:,:,0],axis=1)
figV,axV=plt.subplots(figsize=(4,4))
axV.plot(gaverageseries,meaneigval_num,color=cm[0],linewidth=1.5)
axV.plot(gaverageseries,meaneigval_theo,color=cm[1],linewidth=1.5)

plambda=np.polyfit(gaverageseries,meaneigval_num,2)
meaneigval_poly=plambda[0]*gaverageseries**2+plambda[1]*gaverageseries+plambda[2]
axV.plot(gaverageseries,meaneigval_poly,color=cm[0],linestyle='--',linewidth=1.5,alpha=0.5)


In [282]:
'''
## SIMPLE VERSION, WITHOUT EIGENVECTOR
loading in testing data set
    puring data, delete extreme values
    calculate cov PCA
    calculate directions and amplitude
'''
figm,axm=plt.subplots(2,1,figsize=(5,4))
figs,axs=plt.subplots(2,1,figsize=(5,4))
figc,axc=plt.subplots(3,1,figsize=(5,6))
yticks=np.linspace(-0.05,0.05,2)
ylims=[-0.06,0.06]
xticksE=np.linspace(-0.05,0.10,2)
xlimsE=[-0.06,0.11]
xticksI=np.linspace(-0.13,0.02,2)
xlimsI=[-0.14,0.03]


cm='by'
# ## Right
# axm[0].plot(gaverageseries[1:],np.diff(np.mean(np.abs(statesReig[:,:,0,0]),axis=1)),color=cm[1],linewidth=1.5)
# axm[1].plot(gaverageseries[1:],np.diff(np.mean(np.abs(statesReig[:,:,1,0]),axis=1)),color=cm[1],linewidth=1.5)
# axs[0].plot(gaverageseries,np.mean(statesReig[:,:,0,1],axis=1),color=cm[1],linewidth=1.5)
# axs[1].plot(gaverageseries,np.mean(statesReig[:,:,1,1],axis=1),color=cm[1],linewidth=1.5)
## Left
axm[0].plot(gaverageseries[1:],np.diff(np.mean(statesLeig[:,:,0,0],axis=1)),color=cm[0],linewidth=1.5)
axm[1].plot(gaverageseries[1:],np.diff(np.mean(statesLeig[:,:,1,0],axis=1)),color=cm[0],linewidth=1.5)
axs[0].plot(gaverageseries,np.mean(statesLeig[:,:,0,1],axis=1),color=cm[0],linewidth=1.5)
axs[1].plot(gaverageseries,np.mean(statesLeig[:,:,1,1],axis=1),color=cm[0],linewidth=1.5)
# axs[1].plot(gaverageseries,np.mean(statesLeig[:,:,2,1],axis=1),color='black',linestyle='--',linewidth=1.5)
axc[0].plot(gaverageseries,np.mean(coveigvecAmseries[:,:,0],axis=1),color=cm[1],linewidth=1.5)
axc[1].plot(gaverageseries,np.mean(coveigvecAmseries[:,:,1],axis=1),color=cm[1],linewidth=1.5)
axc[2].plot(gaverageseries,np.mean(coveigvecAmseries[:,:,2],axis=1),color=cm[1],linestyle='--',linewidth=1.5)
# axc[0].plot(gaverageseries[1:],np.diff(np.mean(coveigvecseries[:,:,0],axis=1)),color=cm[1],linewidth=1.5)
# axc[1].plot(gaverageseries[1:],np.diff(np.mean(coveigvecseries[:,:,1],axis=1)),color=cm[1],linewidth=1.5)
# axc[2].plot(gaverageseries,np.mean(coveigvecseries[:,:,2],axis=1),color=cm[1],linestyle='--',linewidth=1.5)


In [150]:
'''
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvector \mu(derivative of l),\sigma(derivative of l), \mu(derivative of r),\sigma(derivative of r)
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leigvec_num,std_perturb_Reigvec_num =np.zeros(2),np.zeros(2) # Excitatory, Inhibitory
std_perturb_Leigvec_num[0],std_perturb_Reigvec_num[0] = np.mean(np.std(np.squeeze(Leigvecseries[ngavg-1,:,:NE,0]),axis=1)),np.mean(np.std(np.squeeze(Reigvecseries[ngavg-1,:,:NE,0]),axis=1))
std_perturb_Leigvec_num[1],std_perturb_Reigvec_num[1] = np.mean(np.std(np.squeeze(Leigvecseries[ngavg-1,:,NE:,0]),axis=1)),np.mean(np.std(np.squeeze(Reigvecseries[ngavg-1,:,NE:,0]),axis=1))
std_perturb_Leigvec_theo,std_perturb_Reigvec_theo =np.zeros(2),np.zeros(2) 
VarhomoR,VarhomoL = 1/(NE*2),2*(JE**2+JI**2)/(NE*2*(JE-JI)**2)
std_perturb_Reigvec_theo[0],std_perturb_Reigvec_theo[1] = (1+xei**2)/2*VarhomoR,(xie**2+xii**2)/2*VarhomoR
std_perturb_Leigvec_theo[0],std_perturb_Leigvec_theo[1] = (2*JE**2*xee**2+2*JI**2*xei**2)/(NE*2*(JE-JI)**2),(2*JE**2*xie**2+2*JI**2*xii**2)/(NE*2*(JE-JI)**2)
std_perturb_Reigvec_theo,std_perturb_Leigvec_theo=np.sqrt(std_perturb_Reigvec_theo),np.sqrt(std_perturb_Leigvec_theo)
''' S inverse term '''
std_perturb_Reigvec_theo=std_perturb_Reigvec_theo/-eigvAm[0]
std_perturb_Leigvec_theo=std_perturb_Leigvec_theo/-eigvAm[0]

print('std of perturbation left vector(E/I), num:',std_perturb_Leigvec_num,'; theo:',std_perturb_Leigvec_theo)
print('std of perturbation right vector(E/I), num:',std_perturb_Reigvec_num,'; theo:',std_perturb_Reigvec_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leigvec_num_,std_perturb_Reigvec_num_ =np.zeros((ngavg,2)),np.zeros((ngavg,2)) # Excitatory, Inhibitory
std_perturb_Leigvec_num_[:,0],std_perturb_Leigvec_num_[:,1] = np.mean(np.squeeze(np.std(np.squeeze(Leigvecseries[:,:,:NE,0]),axis=2)),axis=1),np.mean(np.squeeze(np.std(np.squeeze(Leigvecseries[:,:,NE:,0]),axis=2)),axis=1)
std_perturb_Reigvec_num_[:,0],std_perturb_Reigvec_num_[:,1] = np.mean(np.squeeze(np.std(np.squeeze(Reigvecseries[:,:,:NE,0]),axis=2)),axis=1),np.mean(np.squeeze(np.std(np.squeeze(Reigvecseries[:,:,NE:,0]),axis=2)),axis=1)

fig,ax=plt.subplots(1,2,figsize=(6,3))
ax[0].plot(gaverageseries,std_perturb_Leigvec_num_[:,0],color='red',linewidth=1.5,label=r'$\sigma_{\mathbf{l}}^E')
ax[0].plot(gaverageseries,std_perturb_Leigvec_num_[:,1],color='blue',linewidth=1.5,label=r'$\sigma_{\mathbf{l}}^I')
pLE=np.polyfit(gaverageseries,std_perturb_Leigvec_num_[:,0],1)
pLI=np.polyfit(gaverageseries,std_perturb_Leigvec_num_[:,1],1)
print('var delta Left vectors numerical:',pLE[0],', ',pLI[0])
ax[1].plot(gaverageseries,std_perturb_Reigvec_num_[:,0],color='red',linewidth=1.5,label=r'$\sigma_{\mathbf{r}}^E')
ax[1].plot(gaverageseries,std_perturb_Reigvec_num_[:,1],color='blue',linewidth=1.5,label=r'$\sigma_{\mathbf{r}}^I')
pLE=np.polyfit(gaverageseries,std_perturb_Leigvec_num_[:,0],1)
pLE=np.polyfit(gaverageseries,std_perturb_Leigvec_num_[:,0],1)

std of perturbation left vector(E/I), num: [0.02522    0.02534623] ; theo: [-0.02377674-0.j -0.02377674-0.j]
std of perturbation right vector(E/I), num: [0.01231713 0.01237276] ; theo: [-0.01154701-0.j -0.01154701-0.j]
var delta Left vectors numerical: 0.025619415723140648 ,  0.02573635078615605


In [280]:
'''
The noise covariance -- for loading space
'''
idxtrial=3
lgf,rgf = np.squeeze(Leigvecseries[-1,:,:,0]),np.squeeze(Reigvecseries[-1,:,:,0])
ttmeanE,ttmeanI = np.sum(lgf[idxtrial,:NE]*rgf[idxtrial,:NE]),np.sum(lgf[idxtrial,NE:]*rgf[idxtrial,NE:])
meanE,meanI = meanLeigvec_theo[-1,0]*meanReigvec_theo[-1,0]*NE,meanLeigvec_theo[-1,1]*meanReigvec_theo[-1,1]*NE
noisecovE,noisecovI = ttmeanE-meanE,ttmeanI-meanI
print('theo noise covariance:',noisecovE,noisecovI)

### numerical solution
# coveigvecseries[idxeta,iktrial,0]
noisecovE_num,noisecovI_num = np.mean(np.squeeze(coveigvecseries[-1,:,0])),np.mean(np.squeeze(coveigvecseries[-1,:,1]))
print('num noise covariance:',noisecovE_num,noisecovI_num)
print('total arrangement:',ttmeanE/ttmeanI,JE/JI)


theo noise covariance: 0.09506651777910435 -0.06332251777909365
num noise covariance: (0.14660817906858897+0j) (-0.11574845894836971+0j)
total arrangement: -3.219140488121495 3.5


In [238]:
print(etaset[0]-etaset[1],etaset[2]-etaset[1])
a,b=(etaset[0]-etaset[1])**2,(etaset[2]-etaset[1])**2
print((a+b),np.sqrt(a*b))
print((etaset[0]*etaset[2])/np.sqrt(etaset[0]**2+etaset[2]**2))

## TEST R1:NE L1:NE
idxm,idxc=-10,88
Esum_,Isum_ = np.mean(X1[:NE,NE+idxm+idxc]*Y1[:NE,NE+idxm]),np.mean(X1[NE:,NE+idxm+idxc]*Y1[NE:,NE+idxm])
Esum,Isum = np.mean(X1[:NE,NE+idxm]*Y1[:NE,NE+idxm]),np.mean(X1[NE:,NE+idxm]*Y1[NE:,NE+idxm])
print('rearrage:',Esum,Isum)
print('rearrage change:',Esum_,Isum_)
np.mean(X1[:,12]*Y1[:,6])

-0.15999999999999992 -0.5599999999999999
0.3391999999999999 0.08959999999999994
0.3577708763999664
rearrage: (0.028715780586678605+0j) (-0.027382447254399058+0j)
rearrage change: (-0.027382423091404044+0j) (0.027382422977177835+0j)


(-0.00159375-0.00025260416666666666j)

In [178]:
cm='by'
fig,ax=plt.subplots(2,1,figsize=(5,4))
ax[0].plot(gaverageseries,(np.mean(statesReig[:,:,0,0],axis=1)),color=cm[1],linewidth=1.5)
theo_org=1/np.sqrt(2*NE)-gaverageseries**2*etaset[0]/2.0/eigvAm[0]**2/np.sqrt(NE*2)
ax[0].plot(gaverageseries,theo_org,linestyle='--',linewidth=1.5)

fig,ax=plt.subplots(2,1,figsize=(5,4))
ax[0].plot(gaverageseries,(np.mean(statesLeig[:,:,0,0],axis=1)),color=cm[1],linewidth=1.5)
theo_org=2*JE/np.sqrt(2*NE)/(JE-JI)-gaverageseries**2*etaset[0]*JE/eigvAm[0]**2/np.sqrt(NE*2)/(JE-JI)
ax[0].plot(gaverageseries,theo_org,linestyle='--',linewidth=1.5)
ax[1].plot(gaverageseries,(np.mean(statesLeig[:,:,1,0],axis=1)),color=cm[1],linewidth=1.5)
theo_org=-2*JI/np.sqrt(2*NE)/(JE-JI)+gaverageseries**2*etaset[0]*JI/eigvAm[0]**2/np.sqrt(NE*2)/(JE-JI)
ax[1].plot(gaverageseries,theo_org,linestyle='--',linewidth=1.5)

fig,ax=plt.subplots(figsize=(5,4))
ax.plot(gaverageseries,(np.mean(eigvalseries[:,:,0],axis=1)),color=cm[1],linewidth=1.5)
theo_org=eigvAm[0]+etaset[1]*gaverageseries**2/eigvAm[0]
ax.plot(gaverageseries,theo_org,color=cm[1],linewidth=1.5)



# Backup Code

In [ ]:

# # random gain and symmetricity
# gaverage=1.0
# # homogeneous 
# grand=np.ones(4)*gaverage/np.sqrt(2)
# ''' test r_g and gbar'''
# # theoretical M (grandom) matrix 
# xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
# gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# # first do not multiply at
# gaverage_=0
# for i in range(2):
#     for j in range(2):
#         gaverage_+=gmat[i,j]/2 # ntype=2
# gaverage_=np.sqrt(gaverage_)
# eigvgm,eigvecgm=la.eig(gmat) 
# r_g2=np.max(eigvgm)
# radius_g = np.sqrt(r_g2)
# print('gbar:,',gaverage_,'; effective gain:',radius_g)
# '''
# fixed g_bar but r_g changes (randomly insert)
# eta changes and find out the relationships with eta, rotation and eigenvalue outliers
# * symetaseries 
# * gaverages =1.0
# * grand fixed 
# * ntrials
# '''

print(1/eigvAm[0]/np.sqrt(2*NE))
print(1/eigvAm[0]/np.sqrt(4*NE))